In [1]:
from qiskit.optimization import QuadraticProgram

To map a problem to the correct input format, the optimization module of Qiskit offers a variety of converters. In this tutorial we’re providing an overview on this functionality. Currently, Qiskit contains the following converters. - InequalityToEquality: converts inequality constraints into equality constraints with additional slack variables. - IntegerToBinary: converts integer variables into binary variables and corresponding coefficients. - LinearEqualityToPenalty: convert equality constraints into additional terms of the object function. - QuadraticProgramToQubo: a wrapper for InequalityToEquality, IntegerToBinary, and LinearEqualityToPenalty for convenience.

### InequalityToEquality

In [5]:
#InequalityToEqualityConverter converts inequality constraints into equality constraints 
#with additional slack variables to remove inequality constraints from QuadraticProgram. 
#The upper bounds and the lower bounds of slack variables will be calculated from the difference between 
#the left sides and the right sides of constraints. 
#Signs of slack variables depend on symbols in constraints such as ≤ and ≥

qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.integer_var(lowerbound=0, upperbound=7, name='z')

qp.maximize(linear={'x': 2, 'y': 1, 'z': 1})
qp.linear_constraint(linear={'x': 1, 'y': 1, 'z': 1}, sense='LE', rhs=5.5,name='xyz_leq')
qp.linear_constraint(linear={'x': 1, 'y': 1, 'z': 1}, sense='GE', rhs=2.5,name='xyz_geq')
#qp.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'x'): -1}, sense='<=', rhs=10, name='quad_geq')

print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 2 x + y + z
Subject To
 xyz_leq: x + y + z <= 5.500000000000
 xyz_geq: x + y + z >= 2.500000000000

Bounds
 0 <= x <= 1
 0 <= y <= 1
       z <= 7

Binaries
 x y

Generals
 z
End



In [6]:
from qiskit.optimization.converters import InequalityToEquality

ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp)
print(qp_eq.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 2 x + y + z
Subject To
 xyz_leq: x + y + z + xyz_leq@int_slack = 5
 xyz_geq: x + y + z - xyz_geq@int_slack = 3

Bounds
 0 <= x <= 1
 0 <= y <= 1
       z <= 7
       xyz_leq@int_slack <= 5
       xyz_geq@int_slack <= 6

Binaries
 x y

Generals
 z xyz_leq@int_slack xyz_geq@int_slack
End



### IntegerToBinary

In [7]:
from qiskit.optimization.converters import IntegerToBinary

#Does not work if there is a quadratic constraint
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 2 x + y + z@0 + 2 z@1 + 4 z@2
Subject To
 xyz_leq: x + y + z@0 + 2 z@1 + 4 z@2 + xyz_leq@int_slack@0
          + 2 xyz_leq@int_slack@1 + 2 xyz_leq@int_slack@2 = 5
 xyz_geq: x + y + z@0 + 2 z@1 + 4 z@2 - xyz_geq@int_slack@0
          - 2 xyz_geq@int_slack@1 - 3 xyz_geq@int_slack@2 = 3

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z@0 <= 1
 0 <= z@1 <= 1
 0 <= z@2 <= 1
 0 <= xyz_leq@int_slack@0 <= 1
 0 <= xyz_leq@int_slack@1 <= 1
 0 <= xyz_leq@int_slack@2 <= 1
 0 <= xyz_geq@int_slack@0 <= 1
 0 <= xyz_geq@int_slack@1 <= 1
 0 <= xyz_geq@int_slack@2 <= 1

Binaries
 x y z@0 z@1 z@2 xyz_leq@int_slack@0 xyz_leq@int_slack@1 xyz_leq@int_slack@2
 xyz_geq@int_slack@0 xyz_geq@int_slack@1 xyz_geq@int_slack@2
End



### LinearEqualityToPenalty - formation of a QUBO

In [6]:
from qiskit.optimization.converters import LinearEqualityToPenalty

lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 178 x + 177 y + 177 z@0 + 354 z@1 + 708 z@2 + 110 xyz_leq@int_slack@0
      + 220 xyz_leq@int_slack@1 + 220 xyz_leq@int_slack@2
      - 66 xyz_geq@int_slack@0 - 132 xyz_geq@int_slack@1
      - 198 xyz_geq@int_slack@2 + [ - 44 x^2 - 88 x*y - 88 x*z@0 - 176 x*z@1
      - 352 x*z@2 - 44 x*xyz_leq@int_slack@0 - 88 x*xyz_leq@int_slack@1
      - 88 x*xyz_leq@int_slack@2 + 44 x*xyz_geq@int_slack@0
      + 88 x*xyz_geq@int_slack@1 + 132 x*xyz_geq@int_slack@2 - 44 y^2 - 88 y*z@0
      - 176 y*z@1 - 352 y*z@2 - 44 y*xyz_leq@int_slack@0
      - 88 y*xyz_leq@int_slack@1 - 88 y*xyz_leq@int_slack@2
      + 44 y*xyz_geq@int_slack@0 + 88 y*xyz_geq@int_slack@1
      + 132 y*xyz_geq@int_slack@2 - 44 z@0^2 - 176 z@0*z@1 - 352 z@0*z@2
      - 44 z@0*xyz_leq@int_slack@0 - 88 z@0*xyz_leq@int_slack@1
      - 88 z@0*xyz_leq@int_slack@2 + 44 z@0*xyz_geq@int_slack@0
      + 88 z@0*xyz_geq@int_slack@1 + 132 z@0*x